<a href="https://colab.research.google.com/github/moajjem04/TrafficSignal/blob/main/TrafficSignal_HOG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting up Google Drive

In [10]:
# Mount Google Drive
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT)           # we mount the google drive at /content/drive

/content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Setting up Github

In [17]:
# Clone github repository setup
# import join used to join ROOT path and MY_GOOGLE_DRIVE_PATH
from os.path import join  

# path to your project on Google Drive
MY_GOOGLE_DRIVE_PATH = 'MyDrive/TrafficSignal_1' 
# replace with your Github username 
GIT_USERNAME = "moajjem04" 
# definitely replace with your
GIT_TOKEN = "2c3f8ddbda3d5f1e35b4b69744834379ebd38e06"  
# Replace with your github repository in this case we want 
# to clone deep-learning-v2-pytorch repository
GIT_REPOSITORY = "TrafficSignal" 

PROJECT_PATH = join(ROOT, MY_GOOGLE_DRIVE_PATH)

# It's good to print out the value if you are not sure 
print("PROJECT_PATH: ", PROJECT_PATH)   

# In case we haven't created the folder already; we will create a folder in the project path 
!mkdir "{PROJECT_PATH}"    

#GIT_PATH = "https://{GIT_TOKEN}@github.com/{GIT_USERNAME}/{GIT_REPOSITORY}.git" this return 400 Bad Request for me
GIT_PATH = "https://" + GIT_TOKEN + "@github.com/" + GIT_USERNAME + "/" + GIT_REPOSITORY + ".git"
print("GIT_PATH: ", GIT_PATH)

PROJECT_PATH:  /content/drive/MyDrive/TrafficSignal_1
GIT_PATH:  https://2c3f8ddbda3d5f1e35b4b69744834379ebd38e06@github.com/moajjem04/TrafficSignal.git


In [21]:
%cd "{PROJECT_PATH}"    
# Change directory to the location defined in project_path
!git clone "{GIT_PATH}" # clone the github repository

/content/drive/MyDrive/TrafficSignal_1
Cloning into 'TrafficSignal'...
remote: Enumerating objects: 5031, done.
remote: Counting objects: 100% (5031/5031), done.
remote: Compressing objects: 100% (5000/5000), done.
remote: Total 5031 (delta 30), reused 5024 (delta 29), pack-reused 0
Receiving objects: 100% (5031/5031), 30.12 MiB | 9.16 MiB/s, done.
Resolving deltas: 100% (30/30), done.
Checking out files: 100% (5002/5002), done.


In [ ]:
# !git clone "{GIT_PATH}" ./temp      # clone github repository to temp folder
# !mv ./temp/* "{PROJECT_PATH}"       # move all files/folders in temp folder to folder defined in project path
# !rm -rf ./temp                      # remove all the files/folders in temp folder
# !rsync -aP --exclude=data/ "{PROJECT_PATH}"/*  ./   # use remote sync to copy from google drive to local runtime google colab
#                                                     # but exclude data folder
#                                                     # https://www.computerhope.com/unix/rsync.htm


### Changing Directory to the Github Folder

In [22]:
%pwd

'/content/drive/My Drive/TrafficSignal_1'

In [23]:
%cd TrafficSignal

/content/drive/My Drive/TrafficSignal_1/TrafficSignal


In [24]:
%pwd

'/content/drive/My Drive/TrafficSignal_1/TrafficSignal'

### Visualizing Images

In [25]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np

In [ ]:
from skimage import data, color, feature, transform
import skimage.data

image = color.rgb2gray(data.chelsea())
# hog_vec, hog_vis = feature.hog(image,pixels_per_cell=(16, 16),
#                    cells_per_block=(1, 1), visualize=True)
hog_vec, hog_vis = feature.hog(image,visualize=True)

fig, ax = plt.subplots(1, 2, figsize=(12, 6),
                       subplot_kw=dict(xticks=[], yticks=[]))
ax[0].imshow(image, cmap='gray')
ax[0].set_title('input image')

ax[1].imshow(hog_vis)
ax[1].set_title('visualization of HOG features');

In [74]:
hog_vec.size

153090

In [45]:
%pwd

'/content/drive/My Drive/TrafficSignal_1/TrafficSignal'

In [ ]:
import cv2 as cv
from skimage import io
image = cv.imread('training/00001/00000_00000.ppm')

fig, ax = plt.subplots()
ax.imshow(image)
ax.axis('off')  # clear x-axis and y-axis
plt.show()


# Feature Extraction from Training Images

In [ ]:
count =0;
train_image_num = 400;
train_class =10;
trainingFeatures = np.zeros(shape=(train_image_num*train_class,324))
trainingLabels = np.zeros(shape=(train_image_num*train_class,1))
for nNumFolder in range(1,11):
  print(nNumFolder)
  sFolder = "{0:0=5d}".format(nNumFolder) # Format nNumFolder to folder names
  print(sFolder)
  path = 'training/' + sFolder + '/*.ppm' 
  for file in glob.glob(path):
   #print(file)
   Img= cv.imread(file)
   Img = transform.resize(Img,(32,32))
   trainingFeatures[count,:] = feature.hog(Img,pixels_per_cell=(8, 8),
                                           cells_per_block=(3, 3))
   trainingLabels[count,:] = nNumFolder
   count = count+1;

print(trainingLabels.shape)
print(trainingFeatures.shape)


# Feature Extraction from Testing Images

In [ ]:
count =0;
test_image_num = 100;
test_class =10;
testingFeatures = np.zeros(shape=(test_image_num*test_class,324))
testingLabels = np.zeros(shape=(test_image_num*test_class,1))
for nNumFolder in range(1,11):
  print(nNumFolder)
  sFolder = "{0:0=5d}".format(nNumFolder) # Format nNumFolder to folder names
  print(sFolder)
  path = 'testing/' + sFolder + '/*.ppm' 
  for file in glob.glob(path):
   #print(file)
   Img= cv.imread(file)
   Img = transform.resize(Img,(32,32))
   testingFeatures[count,:] = feature.hog(Img,pixels_per_cell=(8, 8),
                                           cells_per_block=(3, 3))
   testingLabels[count,:] = nNumFolder
   count = count+1;

print(testingLabels.shape)
print(testingFeatures.shape)


In [90]:
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import cross_val_score, GridSearchCV

In [95]:
lin_clf = LinearSVC()
lin_clf.fit(trainingFeatures,trainingLabels)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [106]:
Y_pred = lin_clf.predict(testingFeatures)

In [107]:
# Making the Confusion Matrix
#print(pd.crosstab(Y_test_label, Y_pred_label, rownames=['Actual Activity'], colnames=['Predicted Activity']))
print(confusion_matrix(testingLabels,Y_pred))
print("\n")
print(classification_report(testingLabels,Y_pred))

print("Training set score for SVM: %f" % lin_clf.score(trainingFeatures , trainingLabels))
print("Testing  set score for SVM: %f" % lin_clf.score(testingFeatures  , testingLabels ))

lin_clf.score

[[ 63  20  10   1   1   4   0   0   1   0]
 [  6  77   8   4   1   1   0   0   1   2]
 [ 41   5  52   2   0   0   0   0   0   0]
 [  0   0   3  97   0   0   0   0   0   0]
 [  0   0   0   0 100   0   0   0   0   0]
 [  0   0   0   0   0 100   0   0   0   0]
 [  0   0   0   0   0   0 100   0   0   0]
 [  0   0   0   0   0   0   0 100   0   0]
 [  0   0   0   0   0   0   0   0 100   0]
 [  0   0   0   0   0   0   0   0   0 100]]


              precision    recall  f1-score   support

         1.0       0.57      0.63      0.60       100
         2.0       0.75      0.77      0.76       100
         3.0       0.71      0.52      0.60       100
         4.0       0.93      0.97      0.95       100
         5.0       0.98      1.00      0.99       100
         6.0       0.95      1.00      0.98       100
         7.0       1.00      1.00      1.00       100
         8.0       1.00      1.00      1.00       100
         9.0       0.98      1.00      0.99       100
        10.0       0.98   

<bound method ClassifierMixin.score of LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)>

# Hyperparameter Tuning

In [100]:
#Libraries to Build Ensemble Model : Random Forest Classifier 
# Create the parameter grid based on the results of random search 
params_grid = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

In [ ]:
# Performing CV to tune parameters for best SVM fit 
svm_model = GridSearchCV(SVC(), params_grid, cv=5)
svm_model.fit(trainingFeatures,trainingLabels)

In [103]:
# View the accuracy score
print('Best score for training data:', svm_model.best_score_,"\n") 

# View the best parameters for the model found using grid search
print('Best C:',svm_model.best_estimator_.C,"\n") 
print('Best Kernel:',svm_model.best_estimator_.kernel,"\n")
print('Best Gamma:',svm_model.best_estimator_.gamma,"\n")

final_model = svm_model.best_estimator_
Y_pred = final_model.predict(testingFeatures)

Best score for training data: 0.907 

Best C: 1 

Best Kernel: linear 

Best Gamma: scale 



In [105]:
# Making the Confusion Matrix
#print(pd.crosstab(Y_test_label, Y_pred_label, rownames=['Actual Activity'], colnames=['Predicted Activity']))
print(confusion_matrix(testingLabels,Y_pred))
print("\n")
print(classification_report(testingLabels,Y_pred))

print("Training set score for SVM: %f" % final_model.score(trainingFeatures , trainingLabels))
print("Testing  set score for SVM: %f" % final_model.score(testingFeatures  , testingLabels ))

final_model.score

[[ 68  17  11   0   0   2   0   0   2   0]
 [ 14  73  13   0   0   0   0   0   0   0]
 [ 41   3  56   0   0   0   0   0   0   0]
 [  0   0   1  99   0   0   0   0   0   0]
 [  0   0   0   0 100   0   0   0   0   0]
 [  0   1   0   0   0  99   0   0   0   0]
 [  0   0   0   0   0   0 100   0   0   0]
 [  0   0   0   0   0   0   0 100   0   0]
 [  0   0   0   0   0   0   0   0 100   0]
 [  0   2   0   0   0   0   0   0   0  98]]


              precision    recall  f1-score   support

         1.0       0.55      0.68      0.61       100
         2.0       0.76      0.73      0.74       100
         3.0       0.69      0.56      0.62       100
         4.0       1.00      0.99      0.99       100
         5.0       1.00      1.00      1.00       100
         6.0       0.98      0.99      0.99       100
         7.0       1.00      1.00      1.00       100
         8.0       1.00      1.00      1.00       100
         9.0       0.98      1.00      0.99       100
        10.0       1.00   

<bound method ClassifierMixin.score of SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)>